In [1]:
# Importing the libraries and data files

import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

import tensorflow.data as tf_data
import tensorflow.strings as tf_strings
from sklearn.metrics import accuracy_score
from tqdm import tqdm


import keras
import keras_nlp
from keras import layers
#from keras import ops
from keras.layers import TextVectorization
import tensorflow.keras.backend as K

from tensorflow.python.keras import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


Using TensorFlow backend


In [2]:
eeg_non_seizure_data = np.load('./eeg_data_non_pre_seizure.npy')
eeg_pre_seizure_data = np.load('./eeg_data_pre_seizure_chl60_pad_60.npy')
eeg_pre_seizure_data_zero_padding = np.load('./eeg_data_pre_seizure_chl60_pad_0.npy')
signal_labels = ['EEG Fp1-REF', 'EEG Fp2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG Fz-REF', 'EEG Cz-REF', 'EEG Pz-REF', 'ECG EKG-REF', 'Resp Effort-REF']

In [3]:
def vizualize_eeg_channels(signal,offset=0.0005):
    # Set the number of channels and the offset between each channel
    np.random.seed(42)
    n_channels = len(signal)
    offset = 0.0005
    
    # Set the left and right margins for the window in which we want to plot the data at
    
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(20, 10))
    
    # Plot each signal with an offset
    for i in range(n_channels-2): # Remove the EKG and Effor
      ax.plot(signal[i] + i * offset, label=signal_labels[i])
    
    # Set the y-axis labels to the signal labels
    ax.set_yticks(np.arange(n_channels-2) * offset)
    ax.set_yticklabels(signal_labels[:-2])
    
    # Set the x-axis label
    ax.set_xlabel("Time")
    
    # Add a legend
    #plt.legend()
    
    # Show the plot
    plt.show()

# One hot encoding

In [4]:
y=[]
for i in range(len(eeg_pre_seizure_data)):
    y.append(np.array([0,1]))
#for i in range(len(eeg_non_seizure_data)):
for i in range(250):
    y.append(np.array([1,0]))
y=np.array(y)

In [5]:
X = np.concatenate((eeg_pre_seizure_data[:,:19], eeg_non_seizure_data[:250,:19]))*100

In [6]:
X.shape

(482, 19, 15104)

In [7]:
def get_second_wise_stats_patient_channel(patient):
    patient_embedding=[]
    for channel in range(19):
        mean=np.mean(np.array(np.split(X[patient][channel],np.arange(int(X.shape[2]/256))*256)[1:]), axis=1)
        std=np.mean(np.array(np.split(X[patient][channel],np.arange(int(X.shape[2]/256))*256)[1:]), axis=1)
        min=np.min(np.array(np.split(X[patient][channel],np.arange(int(X.shape[2]/256))*256)[1:]), axis=1)
        max=np.max(np.array(np.split(X[patient][channel],np.arange(int(X.shape[2]/256))*256)[1:]), axis=1)
        median=np.median(np.array(np.split(X[patient][channel],np.arange(int(X.shape[2]/256))*256)[1:]), axis=1)
        info=np.array([mean,std,min,max,median])
        patient_embedding.append(info)
    patient_embedding=np.array(patient_embedding)
    return np.concatenate(patient_embedding)

In [8]:
X_reduced_len=[]
for i in tqdm(range(len(X))):
    X_reduced_len.append(get_second_wise_stats_patient_channel(i))
X=np.array(X_reduced_len)

100%|██████████| 482/482 [00:04<00:00, 116.79it/s]


In [9]:
X.shape

(482, 95, 59)

In [10]:
X=np.array([np.array([(X[index][channel]-np.min(X[index][channel]))/(np.max(X[index][channel])-np.min(X[index][channel])+0.000001) for channel in range(X.shape[1])]) for index in range(len(X))])

In [11]:
X.shape

(482, 95, 59)

In [12]:
X=np.moveaxis(X,-1,-2)

In [13]:
print(X.shape)
print(y.shape)

(482, 59, 95)
(482, 2)


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.3)

In [15]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(337, 59, 95) (337, 2)
(145, 59, 95) (145, 2)


In [16]:
#import keras.ops as ops
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:,tf.newaxis, tf.newaxis, :], dtype='int32')
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class Token_and_PositionalEmbedding(layers.Layer):
    #def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
    def __init__(self, sequence_length, **kwargs):
        super().__init__(**kwargs)
       
        #self.position_embeddings = layers.Embedding(
        #    input_dim=sequence_length, output_dim=embed_dim
        #)

        self.position_embeddings = keras_nlp.layers.SinePositionEncoding(
           sequence_length, **kwargs
        )

        self.sequence_length = sequence_length
        #self.vocab_size = vocab_size
        #self.embed_dim = embed_dim

    def call(self, inputs):
    
        positional_encoding = self.position_embeddings(inputs)

        return positional_encoding

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        else:
            return tf.math.not_equal(inputs,0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


In [17]:
def weighted_binary_crossentropy(target, output, weights):
    target = tf.convert_to_tensor(target)
    output = tf.convert_to_tensor(output)
    weights = tf.convert_to_tensor(weights, dtype=target.dtype)

    epsilon_ = tf.constant(tf.keras.backend.epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, epsilon_, 1.0 - epsilon_)

    # Compute cross-entropy from probabilities.
    bce = weights[1] * target * tf.math.log(output + epsilon_)
    bce += weights[0] * (1 - target) * tf.math.log(1 - output + epsilon_)
    return -bce

class WeightedBinaryCrossentropy:
    def __init__(
        self,
        label_smoothing=0.0,
        weights = [1.0, 1.0],
        axis=-1,
        name="weighted_binary_crossentropy",
        fn = None,
    ):
        """Initializes `WeightedBinaryCrossentropy` instance.
        Args:
          from_logits: Whether to interpret `y_pred` as a tensor of
            [logit](https://en.wikipedia.org/wiki/Logit) values. By default, we
            assume that `y_pred` contains probabilities (i.e., values in [0,
            1]).
          label_smoothing: Float in [0, 1]. When 0, no smoothing occurs. When >
            0, we compute the loss between the predicted labels and a smoothed
            version of the true labels, where the smoothing squeezes the labels
            towards 0.5.  Larger values of `label_smoothing` correspond to
            heavier smoothing.
          axis: The axis along which to compute crossentropy (the features
            axis).  Defaults to -1.
          name: Name for the op. Defaults to 'weighted_binary_crossentropy'.
        """
        super().__init__()
        self.weights = weights # tf.convert_to_tensor(weights)
        self.label_smoothing = label_smoothing
        self.name = name
        self.fn = weighted_binary_crossentropy if fn is None else fn

    def __call__(self, y_true, y_pred):
        y_pred = tf.convert_to_tensor(y_pred)
        y_true = tf.cast(y_true, y_pred.dtype)
        self.label_smoothing = tf.convert_to_tensor(self.label_smoothing, dtype=y_pred.dtype)

        def _smooth_labels():
            return y_true * (1.0 - self.label_smoothing) + 0.5 * self.label_smoothing

        y_true = tf.__internal__.smart_cond.smart_cond(self.label_smoothing, _smooth_labels, lambda: y_true)

        return tf.reduce_mean(self.fn(y_true, y_pred, self.weights),axis=-1)

    def get_config(self):
        config = {"name": self.name, "weights": self.weights, "fn": self.fn}

        # base_config = super().get_config()
        return dict(list(config.items()))

    @classmethod
    def from_config(cls, config):
        """Instantiates a `Loss` from its config (output of `get_config()`).
        Args:
            config: Output of `get_config()`.
        """
        if saving_lib.saving_v3_enabled():
            fn_name = config.pop("fn", None)
            if fn_name:
                config["fn"] = get(fn_name)
        return cls(**config)

In [18]:
embed_dim = X.shape[2]
latent_dim = 1024
num_heads = 6
signal_len = X.shape[1]
max_hamiltonian_len = 2
hamiltonian_cycle = y_train
def get_model():
    signal = keras.Input(shape=(signal_len,embed_dim), dtype="float64")

    pos_emb_1= Token_and_PositionalEmbedding(signal_len)(signal)

    signal=tf.keras.layers.Add()([signal, pos_emb_1])

    signal_lstm = layers.LSTM(units=256, return_sequences=True, activation="tanh")(signal)

    pos_emb_2= Token_and_PositionalEmbedding(signal_len)(signal_lstm)
    
    signal_pos=tf.keras.layers.Add()([signal_lstm, pos_emb_2])
    
    x_att= keras.layers.MultiHeadAttention(num_heads,embed_dim)(query=signal_pos,value=signal_pos,key=signal_pos)

    x_res1=tf.keras.layers.Add()([x_att, signal_pos])
    x_ln1=layers.LayerNormalization()(x_res1)
    
    x_2 = layers.Dense(latent_dim, activation="relu")(x_ln1)
    x_2=layers.Dense(embed_dim)(x_2)

    x_res2=tf.keras.layers.Add()([x_att, x_ln1])
    x_ln2=layers.LayerNormalization()(x_res2)
    
    x_3 = keras.layers.Flatten()(x_ln2)
    output = keras.layers.Dense(2, activation='sigmoid')(x_3)
    model = keras.Model(inputs=signal, outputs=output)
    return model

transformer_model = get_model()

In [19]:
model=get_model()

In [20]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 59, 95)]     0           []                               
                                                                                                  
 lstm_1 (LSTM)                  (None, 59, 256)      360448      ['input_4[0][0]']                
                                                                                                  
 token_and__positional_embeddin  (None, 59, 256)     0           ['lstm_1[1][0]']                 
 g_3 (Token_and_PositionalEmbed                                                                   
 ding)                                                                                            
                                                                                            

In [21]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
     
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
        val_targ = self.model.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print ( "— val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
        return
 
metrics = Metrics()

In [22]:
@tf.autograph.experimental.do_not_convert
def f1_score(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    return f1


In [23]:
epochs = 100 # This should be at least 30 for convergence
wbce = WeightedBinaryCrossentropy(weights = [1.0, 1.0])
val_data=X_test[:1000]
val_target=y_test[:1000]

#model.compile(
#    keras.optimizers.Adam(learning_rate=0.000001), loss=wbce, metrics=['accuracy']
#)

model.compile(
    keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy"
)

#model.fit(X_train, y_train, validation_data=(val_data,val_target), epochs= epochs, callbacks=[metrics])

model.fit(X_train, y_train, epochs= epochs, shuffle=True, validation_split=0.05)

Epoch 1/100
10/10 [==============================] - 4s 78ms/step - loss: 0.8305 - val_loss: 0.7428
Epoch 2/100
10/10 [==============================] - 0s 22ms/step - loss: 0.7242 - val_loss: 0.7100
Epoch 3/100
10/10 [==============================] - 0s 20ms/step - loss: 0.7091 - val_loss: 0.7046
Epoch 4/100
10/10 [==============================] - 0s 18ms/step - loss: 0.6894 - val_loss: 0.6887
Epoch 5/100
10/10 [==============================] - 0s 24ms/step - loss: 0.6923 - val_loss: 0.7116
Epoch 6/100
10/10 [==============================] - 0s 22ms/step - loss: 0.6846 - val_loss: 0.6804
Epoch 7/100
10/10 [==============================] - 0s 21ms/step - loss: 0.6995 - val_loss: 0.6910
Epoch 8/100
10/10 [==============================] - 0s 22ms/step - loss: 0.7131 - val_loss: 0.6822
Epoch 9/100
10/10 [==============================] - 0s 22ms/step - loss: 0.6881 - val_loss: 0.6716
Epoch 10/100
10/10 [==============================] - 0s 18ms/step - loss: 0.6727 - val_loss: 0.7690

In [24]:
y_pred=model.predict(X_test)

5/5 [==============================] - 0s 7ms/step


In [25]:
from sklearn.metrics import average_precision_score
average_precision_score(y_test, y_pred)

0.6755345867818683

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test[:,1],1*(y_pred[:,1]>0.5))

array([[40, 33],
       [17, 55]], dtype=int64)